In [1]:
import os
from pathlib import Path
from data_maker import DataMaker
model_file = Path('/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/output/multi_task_model_9_13050.pt')
dataMaskDir = Path('/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/MLM/data_mlm/pertured_data/masked_data_json_v2')
dataMaker = DataMaker(
    data_file= os.path.join(dataMaskDir,'pertured_data_alter_full.json'),
    out_dir= os.path.join(dataMaskDir,'data_predictions'),
    saved_model_path=model_file
)

resultwordMasked  = dataMaker.get_predictions(is_masked=True)
resultwordMasked = {
    'uid': resultwordMasked[0],
    'pred': resultwordMasked[1],
    'score': resultwordMasked[2],
    'logitsSoftmax': resultwordMasked[3],
    'logitsRaw': resultwordMasked[4]    
}


/home/phatpham/anaconda3/envs/min_ds-env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-19 15:52:09.735950: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-19 15:52:09.953553: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-19 15:52:11.600370: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: 

Model Name:  bert


100%|██████████| 49/49 [04:24<00:00,  5.41s/it]


In [2]:
len(resultwordMasked['pred'])

1568

In [3]:
dataOriginDir = Path('/mnt/c/Users/Phat Pham/Documents/THESIS/SRLPredictionEasel/MLM/data_mlm/process_folder/mlm_output')
dataMaker = DataMaker(
    data_file= os.path.join(dataOriginDir,'mlm_abolish_full.json'),
    out_dir= os.path.join(dataOriginDir, 'data_predictions'),
    saved_model_path=model_file
)

resultwordOrigin  = dataMaker.get_predictions(is_masked=False)
resultwordOrigin = {
    'uid': resultwordOrigin[0],
    'pred': resultwordOrigin[1],
    'score': resultwordOrigin[2],
    'logitsSoftmax': resultwordOrigin[3],
    'logitsRaw': resultwordOrigin[4]    
}


Model Name:  bert


100%|██████████| 32/32 [02:09<00:00,  4.04s/it]


In [ ]:
{'B-A1': 0,
 'I-A1': 1,
 'O': 2,
 'B-V': 3,
 'B-A0': 4,
 'I-A0': 5,
 'B-A4': 6,
 'I-A4': 7,
 'I-A3': 8,
 'B-A2': 9,
 'I-A2': 10,
 'B-A3': 11,
 '[CLS]': 12,
 '[SEP]': 13,
 'X': 14}

In [10]:
import torch


def get_idx_arg_preds(preds_origin, preds_masked, label_origin=None): # label_origin: nhãn gold
    list_idx_arg_change = []
    print(len(preds_origin), len(preds_masked ))
    assert len(preds_origin) == len(preds_masked), 'Length of preds_origin and preds_masked must be the same'
    for i in range(len(preds_origin)):
        test = preds_origin[i] not in [2, 3, 12, 13, 14] and preds_masked[i] not in [2, 3, 12, 13, 14]
        if label_origin:
            assert len(preds_origin) == len(label_origin), 'Length of preds_origin and label_origin must be the same'
            if test or label_origin[i] not in [2, 3, 12, 13, 14]:
                list_idx_arg_change.append(i)
        else:
            if test:
                list_idx_arg_change.append(i)
    return list_idx_arg_change

def calculateInfluenceScore(outLogitsSigmoid_original, outLogitsSigmoid_meddle, list_arg_change):
    influence_score = []
    weight = []
    assert len(outLogitsSigmoid_original) == len(outLogitsSigmoid_meddle) 
    for i in range(len(outLogitsSigmoid_original)):
        if i not in list_arg_change:
            continue
        max_index_original = torch.argmax(outLogitsSigmoid_original[i])
        max_index_meddle = torch.argmax(outLogitsSigmoid_meddle[i])
        if max_index_original == max_index_meddle:
            influence_score.append((outLogitsSigmoid_original[i][max_index_original] - outLogitsSigmoid_meddle[i][max_index_meddle]) / max(outLogitsSigmoid_original[i][max_index_original], outLogitsSigmoid_meddle[i][max_index_meddle]))
            weight.append(1)
        else:
            influ_old_label = (outLogitsSigmoid_original[i][max_index_original] - outLogitsSigmoid_meddle[i][max_index_original]) / max(outLogitsSigmoid_original[i][max_index_original], outLogitsSigmoid_meddle[i][max_index_original])
            influ_new_label = (outLogitsSigmoid_meddle[i][max_index_meddle] - outLogitsSigmoid_original[i][max_index_meddle]) / max(outLogitsSigmoid_original[i][max_index_meddle], outLogitsSigmoid_meddle[i][max_index_meddle])
            influence_score.append(influ_old_label + influ_new_label)
            weight.append(2)
    return influence_score, weight

In [2]:
l = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
sum(l)

39

In [1]:
print(resultwordOrigin['uid'][666], resultwordMasked['uid'][683])

NameError: name 'resultwordOrigin' is not defined

In [17]:
import torch
for i in range(len(resultwordOrigin['logitsSoftmax'])):
    resultwordOrigin['logitsSoftmax'][i] = torch.tensor(resultwordOrigin['logitsSoftmax'][i])
for i in range(len(resultwordMasked['logitsSoftmax'])):
    resultwordMasked['logitsSoftmax'][i] = torch.tensor(resultwordMasked['logitsSoftmax'][i])
    
influenceScore, weight = calculateInfluenceScore(resultwordOrigin['logitsSoftmax'][666], resultwordMasked['logitsSoftmax'][683], get_idx_arg_preds(resultwordOrigin['pred'][666], resultwordMasked['pred'][683]))

17 57


/tmp/ipykernel_13662/2814686101.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  resultwordOrigin['logitsSoftmax'][i] = torch.tensor(resultwordOrigin['logitsSoftmax'][i])
/tmp/ipykernel_13662/2814686101.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  resultwordMasked['logitsSoftmax'][i] = torch.tensor(resultwordMasked['logitsSoftmax'][i])


AssertionError: Length of preds_origin and preds_masked must be the same